In [8]:
#Imports
import torch
import os
import pandas as pd
import numpy as np
import math

# Data preprocessing
This part of the code is to generate 20x20 matrices representing the distance between joints in all frames.
This resulted in two .npy files X_train.npy and Y_train.npy
X_train contains a list of all 20x20 matrices, and Y_train a list of 1x6 vectors representing the correct letter.

Note: We will have to re-run this when the additional data for the other letters is published.

In [9]:
#Loop through directories in dataset_v0
path_of_data_directory = "../dataset_v0/"

annotations_A = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_A/annotations.csv"))
annotations_B = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_B/annotations.csv"))
annotations_C = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_C/annotations.csv"))
annotations_L = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_L/annotations.csv"))
annotations_R = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_R/annotations.csv"))
annotations_U = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_U/annotations.csv"))


annotations_old = [annotations_A, annotations_B, annotations_C, annotations_L, annotations_R, annotations_U]

print(annotations_A.shape)
print(annotations_B.shape)
print(annotations_C.shape)
print(annotations_L.shape)
print(annotations_R.shape)
print(annotations_U.shape)

# G H I V W Y är på fel form

(147819, 9)
(150003, 9)
(146118, 9)
(152229, 9)
(154518, 9)
(156660, 9)


In [10]:
#Loop through directories in dataset_v0
path_of_data_directory = "../dataset_v0/"

annotations_G = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_G/annotations.csv"))
annotations_H = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_H/annotations.csv"))
annotations_I = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_I/annotations.csv"))
annotations_V = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_V/annotations.csv"))
annotations_W = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_W/annotations.csv"))
annotations_Y = pd.read_csv(os.path.join(path_of_data_directory, "ASL_letter_Y/annotations.csv"))

annotations_new = [annotations_G, annotations_H, annotations_I, annotations_V, annotations_W, annotations_Y]

print(annotations_G.shape)
print(annotations_H.shape)
print(annotations_I.shape)
print(annotations_V.shape)
print(annotations_W.shape)
print(annotations_Y.shape)

# G H I V W Y är på fel form

(53193, 7)
(46977, 7)
(44730, 7)
(52647, 7)
(47145, 7)
(53361, 7)


In [11]:
#Sort the new data as the old one is sorted
sorter = ['root', 'thumb_1', 'thumb_2', 'thumb_3', 'index_1','index_2','index_3','index_4','middle_1','middle_2','middle_3','middle_4',
'ring_1','ring_2','ring_3','ring_4','pinky_1','pinky_2','pinky_3','pinky_4']
sorterIndex = dict(zip(sorter, range(len(sorter))))
sorted_annotations = []
for annotation in annotations_new:

        annotation['Rank'] = annotation['joint'].map(sorterIndex)

        annotation.sort_values(['frame', 'person_idx', 'Rank', ],
                ascending = [True, True, True], inplace = True)
        annotation.drop('Rank', 1, inplace = True)
        sorted_annotations.append(annotation)

print(annotations_new[1])

C:\Users\Jacob\AppData\Local\Temp\ipykernel_19768\3191316436.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  annotation.drop('Rank', 1, inplace = True)


            gesture  frame          joint           x           y  person_idx  \
43177  ASL_letter_H      0           root    0.000000    0.000000       43460   
43367  ASL_letter_H      0        thumb_1    0.000000    0.000000       43460   
43557  ASL_letter_H      0        thumb_2    0.000000    0.000000       43460   
43747  ASL_letter_H      0        thumb_3    0.000000    0.000000       43460   
43937  ASL_letter_H      0        index_1  363.683333   77.541667       43460   
...             ...    ...            ...         ...         ...         ...   
39935  ASL_letter_H    288        pinky_1  223.304427   69.117773       92618   
40224  ASL_letter_H    288        pinky_2  222.237760  148.051107       92618   
40513  ASL_letter_H    288        pinky_3    0.000000    0.000000       92618   
40802  ASL_letter_H    288        pinky_4    0.000000    0.000000       92618   
35022  ASL_letter_H    288  hand_position    0.000000    0.000000       92618   

       video_idx  
43177   

In [12]:
#Merge the two annotation lists together
print(len(annotations_old))
print(len(annotations_new))
annotations = annotations_old + annotations_new
print(len(annotations))
for annotation in annotations:
    print(annotation.shape)

6
6
12
(147819, 9)
(150003, 9)
(146118, 9)
(152229, 9)
(154518, 9)
(156660, 9)
(53193, 7)
(46977, 7)
(44730, 7)
(52647, 7)
(47145, 7)
(53361, 7)


In [16]:
#Need to put each video in annotations_old and annotations_new in a new list based on it's video index. Then shuffle!
video_list = []

for annotation in annotations:
    video_idx_counter = 0
    while(video_idx_counter <= np.max(annotation.video_idx)):
        video_temp = annotation[annotation.video_idx == video_idx_counter]
        video_list.append(video_temp)
        #print(video_idx_counter, " against max: " ,np.max(annotation.video_idx))
        video_idx_counter += 1
print("We have", len(video_list), "videos")

We have  280 videos


In [22]:
import random
#Now we want to shuffle these videos
#print(video_list[0])
random.shuffle(video_list)
print(video_list[0])

          ID  Unnamed: 0  frame       gesture          joint  person_idx  \
49203  49203         NaN      0  ASL_letter_A  hand_position       53325   
49204  49204         NaN      0  ASL_letter_A           root       53325   
49205  49205         NaN      0  ASL_letter_A        thumb_1       53325   
49206  49206         NaN      0  ASL_letter_A        thumb_2       53325   
49207  49207         NaN      0  ASL_letter_A        thumb_3       53325   
...      ...         ...    ...           ...            ...         ...   
54301  54301         NaN    242  ASL_letter_A         ring_4       53325   
54302  54302         NaN    242  ASL_letter_A        pinky_1       53325   
54303  54303         NaN    242  ASL_letter_A        pinky_2       53325   
54304  54304         NaN    242  ASL_letter_A        pinky_3       53325   
54305  54305         NaN    242  ASL_letter_A        pinky_4       53325   

       video_idx           x           y  
49203         11    0.000000    0.000000  
4

In [23]:
annotations = video_list
#Separate each frame into its own object and put them in a list
list_of_frames = []
for annotation in annotations:
    #Removing hand position rows from data (is always 0?)
    annotation = annotation[annotation.joint != 'hand_position']
    i = 0
    #Seperate each frame and add to a list
    while i < len(annotation):
        frame = annotation[i : i+20]
        list_of_frames.append(frame)
        i += 20


In [30]:
print("Number of unique frames:", len(list_of_frames))
#print("Example of frame structure:", list_of_frames[20])
test_frame = list_of_frames[3000]
print(test_frame)

Number of unique frames: 57400
          ID  Unnamed: 0  frame       gesture     joint  person_idx  \
70498  70498         NaN    107  ASL_letter_C      root      192011   
70499  70499         NaN    107  ASL_letter_C   thumb_1      192011   
70500  70500         NaN    107  ASL_letter_C   thumb_2      192011   
70501  70501         NaN    107  ASL_letter_C   thumb_3      192011   
70502  70502         NaN    107  ASL_letter_C   index_1      192011   
70503  70503         NaN    107  ASL_letter_C   index_2      192011   
70504  70504         NaN    107  ASL_letter_C   index_3      192011   
70505  70505         NaN    107  ASL_letter_C   index_4      192011   
70506  70506         NaN    107  ASL_letter_C  middle_1      192011   
70507  70507         NaN    107  ASL_letter_C  middle_2      192011   
70508  70508         NaN    107  ASL_letter_C  middle_3      192011   
70509  70509         NaN    107  ASL_letter_C  middle_4      192011   
70510  70510         NaN    107  ASL_letter_C 

In [32]:
#Function that returns the Y-vector for a given gestures i.e [1 0 0 0 0 0] for the letter A
gestures = ["ASL_letter_A", "ASL_letter_B", "ASL_letter_C", "ASL_letter_L", "ASL_letter_R", "ASL_letter_U",
"ASL_letter_G", "ASL_letter_H", "ASL_letter_I", "ASL_letter_V", "ASL_letter_W", "ASL_letter_Y"]
def getYVectorFromGesture(gesture):
    y = np.zeros(12)
    y[gestures.index(gesture)] = 1
    return y

print(getYVectorFromGesture("ASL_letter_Y"))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [33]:
#This function creates a 20x20 distance matrix for each of the 50k frames. Took me 2 hours /Jacob

# Turn each frame into a distance matrix
X_train = []
Y_train = []
count = 1
#For each frame in our dataset we want to create a distance matrix
for frame in list_of_frames:
    distance_matrix = np.zeros((20,20))

    #For each joint in the frame
    for i in range(len(frame)):
        #Check that the joint is visible
        if(frame.iloc[i].x != 0 and frame.iloc[i].y != 0):
            #For each other joint in the frame
            for j in range(len(frame)):
                #Check that the joint is visible
                if(frame.iloc[j].x != 0 and frame.iloc[j].y != 0):
                    #Calculate distance between joint i and all joints j and add to matrix
                    distance_matrix[i,j] = math.sqrt((frame.iloc[i].x - frame.iloc[j].x)**2 + (frame.iloc[i].y - frame.iloc[j].y)**2)

    #Append the distance matrix for each frame into X_train
    X_train.append(distance_matrix)

    #Get the correct letter from the frame and save it in the format:
    #  [A B C L R U] = [1 0 0 0 0 0] if A is correct
    #  [0 0 1 0 0 0] if C is correct etc. 
    Y_train.append(getYVectorFromGesture(frame.iloc[0].gesture))
    if (count % 250 == 0):
        print("Number of frames done:", count)

    count += 1

#Saves the X_train and Y_train data to numpy file
np.save('all_X.npy', X_train, allow_pickle=True)
np.save('all_Y.npy', Y_train, allow_pickle=True)

Number of frames done: 250
Number of frames done: 500
Number of frames done: 750
Number of frames done: 1000
Number of frames done: 1250
Number of frames done: 1500
Number of frames done: 1750
Number of frames done: 2000
Number of frames done: 2250
Number of frames done: 2500
Number of frames done: 2750
Number of frames done: 3000
Number of frames done: 3250
Number of frames done: 3500
Number of frames done: 3750
Number of frames done: 4000
Number of frames done: 4250
Number of frames done: 4500
Number of frames done: 4750
Number of frames done: 5000
Number of frames done: 5250
Number of frames done: 5500
Number of frames done: 5750
Number of frames done: 6000
Number of frames done: 6250
Number of frames done: 6500
Number of frames done: 6750
Number of frames done: 7000
Number of frames done: 7250
Number of frames done: 7500
Number of frames done: 7750
Number of frames done: 8000
Number of frames done: 8250
Number of frames done: 8500
Number of frames done: 8750
Number of frames done: 

Test on how many frames there are of each gesture

In [6]:
import numpy as np
#Read test from the .npy files containing data
X_train = np.load('./Data/all_X.npy', allow_pickle=True)
Y_train = np.load('./Data/all_Y.npy', allow_pickle=True)

print(np.sum(Y_train))
index = np.zeros(12)
for row in Y_train:
    idx = np.where(row == 1)
    index[idx] += 1
print(index)

57400.0
[7039. 7143. 6958. 7249. 7358. 7460. 2533. 2237. 2130. 2507. 2245. 2541.]
